In [12]:
#import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS
import spacy

In [13]:
df = pd.read_csv('C:\\AA-Classes\\Sem-3\\NLP\\ACTUAL.csv',encoding='latin-1')
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [14]:
df['label'].value_counts()

0    2582
1    2418
Name: label, dtype: int64

In [15]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [16]:
x = df['text']
y = df['label']

In [17]:
punct = string.punctuation
stopwords = list(STOP_WORDS) # list of stopwords

In [18]:
nlp = spacy.load('en_core_web_lg')
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
 
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [19]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

In [20]:
##TF_IDF Vectorizer
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [21]:

#classifier = LinearSVC()
#from sklearn.neighbors import KNeighborsClassifier
lsvc = LinearSVC()
xgboost = XGBClassifier()
mnb = MultinomialNB()
rf = RandomForestClassifier()

In [22]:
classifier_list = [lsvc,xgboost,mnb, rf]

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
x_train.shape, x_test.shape

((4000,), (1000,))

In [25]:
performance_df = pd.DataFrame(index=['Accuracy', 'Precision', 'Recall', 'AUC'])

for classifier in classifier_list:
    clf = Pipeline([('tfidf',tfidf),('clf',classifier)])
    clf.fit(x_train, y_train)
    classifier['y_pred'] = clf.predict(x_test)
    accuracy = round(accuracy_score(y_test, classifier['y_pred']),2)
    precision = round(precision_score(y_test, classifier['y_pred']),2)
    recall = round(recall_score(y_test, classifier['y_pred']),2)
    auc = round(roc_auc_score(y_test, classifier['y_pred']),2)
    classifier_name = classifier['classifier'].__class__.__name__
    performance_df[classifier_name] = [accuracy, precision, recall, auc]

TypeError: 'LinearSVC' object does not support item assignment

((4000,), (1000,))

In [62]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])

In [63]:
# model fit
clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x000002721D706A60>)),
                ('clf', SVC())])

In [64]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = clf.predict(x_test)

In [65]:
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.81      0.83       480
           1       0.83      0.86      0.85       520

    accuracy                           0.84      1000
   macro avg       0.84      0.83      0.84      1000
weighted avg       0.84      0.84      0.84      1000



In [66]:
accuracy_score(y_test, y_pred)

0.836

In [49]:
clf.predict(["worst movie ever"])

array([0])